# Customizing nbconvert

Under the hood, nbconvert uses [Jinja templates](https://jinja2.readthedocs.io/en/latest/intro.html) to specify how the notebooks should be formatted. These templates can be fully customized, allowing you to use nbconvert to create notebooks in different formats with different styles as well.

### Converting a notebook to an (I)Python script and printing to stdout

Out of the box, nbconvert can be used to convert notebooks to plain Python files. For example, the following command converts the `example.ipynb` notebook to Python and prints out the result:

In [7]:
!jupyter nbconvert --to python 'example.ipynb' --stdout

[NbConvertApp] Converting notebook example.ipynb to python

# coding: utf-8

# # Example notebook

# ### Markdown cells
# 
# This is an example notebook that can be converted with `nbconvert` to different formats. This is an example of a markdown cell.

# ### LaTeX Equations
# 
# Here is an equation:
# 
# $$
# y = \sin(x)
# $$

# ### Code cells

# In[1]:

print("This is a code cell that produces some output")


# ### Inline figures

# In[2]:

get_ipython().magic('matplotlib inline')

import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)
plt.plot(x, y)



From the code, you can see that non-code cells are also exported. If you wanted to change that behaviour, you would first look to nbconvert [configuration options page](./config_options.rst) to see if there is an option available that can give you your desired behaviour. 

In this case, if you wanted to remove code cells from the output, you could use the `TemplateExporter.exclude_markdown` traitlet directly, as below. 

In [8]:
!jupyter nbconvert --to python 'example.ipynb' --stdout --TemplateExporter.exclude_markdown=True

[NbConvertApp] Converting notebook example.ipynb to python

# coding: utf-8

# In[1]:

print("This is a code cell that produces some output")


# In[2]:

get_ipython().magic('matplotlib inline')

import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)
plt.plot(x, y)



## Custom Templates 

As mentioned above, if you want to change this behavior, you can use a custom template.  The custom template inherits from the Python template and overwrites the markdown blocks so that they are empty. 

Below is an example of a custom template, which we write to a file called `simplepython.tpl`. This template removes markdown cells from the output, and also changes how the execution count numbers are formatted:

In [15]:
%%writefile simplepython.tpl

{% extends 'python.tpl'%}

## remove markdown cells
{% block markdowncell -%}
{% endblock markdowncell %}

## change the appearance of execution count
{% block in_prompt %}
# [{{ cell.execution_count if cell.execution_count else ' ' }}]:
{% endblock in_prompt %}

Overwriting simplepython.tpl


Using this template, we see that the resulting Python code does not contain anything that was previously in a markdown cell, and has special comments regarding the execution counts:

In [14]:
!jupyter nbconvert --to python 'example.ipynb' --stdout --template=simplepython.tpl

[NbConvertApp] Converting notebook example.ipynb to python


# coding: utf-8

# [1]:

print("This is a code cell that produces some output")


# [2]:

get_ipython().magic('matplotlib inline')

import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)
plt.plot(x, y)



## Template structure

Nbconvert templates consist of a set of nested blocks. When defining a new
template, you extend an existing template by overriding some of the blocks.

All the templates shipped in nbconvert have the basic structure described here,
though some may define additional blocks.

In [4]:
from IPython.display import HTML, display
with open('template_structure.html') as f:
    display(HTML(f.read()))

### A few gotchas

Jinja blocks use `{% %}` by default which does not play nicely with LaTeX, so those are replaced by `((* *))` in LaTeX templates.

## Templates using cell tags

The notebook file format supports attaching arbitrary JSON metadata to each cell. In addition, every cell has a special `tags` metadata field that accepts a list of strings that indicate the cell's tags. To apply these, go to the `View → CellToolbar → Tags` option which will create a Tag editor at the top of every cell. 

First choose a notebook you want to convert to html, and apply the tags: `"Easy"`, `"Medium"`, or 
`"Hard"`.  

With this in place, the notebook can be converted using a custom template.

Design your template in the cells provided below.

Hint: tags are located at `cell.metadata.tags`, the following Python code collects the value of the tag: 

```python
cell['metadata'].get('tags', [])
```

Which you can then use inside a Jinja template as in the following:

In [16]:
%%writefile mytemplate.tpl

{% extends 'full.tpl'%}
{% block any_cell %}
{% if 'Hard' in cell['metadata'].get('tags', []) %}
        {{ super() }}
    </div>
{% elif 'Medium' in cell['metadata'].get('tags', []) %}
    <div style="border:thin solid orange">
        {{ super() }}
    </div>
{% elif 'Easy' in cell['metadata'].get('tags', []) %}
    <div style="border:thin solid green">
        {{ super() }}
    </div>
{% else %}
    {{ super() }}
{% endif %}
{% endblock any_cell %}

Writing mytemplate.tpl


Now, if we collect the result of using nbconvert with this template, and display the resulting html, we see the following:

In [31]:
example = !jupyter nbconvert --to html 'example.ipynb' --template=mytemplate.tpl --stdout
example = example[3:] # have to remove the first three lines which are not proper html
from IPython.display import HTML, display
display(HTML('\n'.join(example))) 

## Templates using custom cell metadata 

We demonstrated above how to use cell tags in a template to apply custom styling to a notebook. But remember, the notebook file format supports attaching _arbitrary_ JSON metadata to each cell, not only cell tags. 
Here, we describe an exercise for using a `example.difficulty` metadata field (rather than cell tags) to indicate how difficult it is to understand different cells in a notebook.

### Dealing with missing metadata fields

One of the challenges of dealing with custom metadata is to handle its nested structure in a way that does not rely on the custom metadata being present in every cell.




Hint: if your tags are located at `cell.metadata.example.difficulty`, the following Python code would get the value of the tag:


```python
cell['metadata'].get('example', {}).get('difficulty', '')
```

The following lines of code may be a helpful starting point:

In [ ]:
jupyter nbconvert --to html <your notebook.ipynb> --template=mytemplate.tpl
